# Chapter 2: First AI Agent with LangChain

## Objectives
By the end of this notebook, you will:
- Understand what a **Large Language Model (LLM)** is and how it generates text.
- Know the difference between a raw **LLM** and an **AI Agent**.
- Structure a **prompt** (system / user).
- Configure a model and control its **temperature**.
- Chain simple components into a **LangChain pipeline**.


In [9]:
# ===============================================================
# 0. Basic setup
# ===============================================================
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda


True

## 1. Connecting to Groq (Basic Setup)

Before we use any model, we need to connect to Groq’s API.

1. Create a `.env` file in your project root with `GROQ_API_KEY=your_api_key_here`
2. Make sure this file is **ignored by Git** (`.gitignore`).
3. Load your environment variables and test the connection below.


In [18]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
# Check if the key is loaded correctly
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("❌ GROQ_API_KEY not found. Please create your .env file.")
else:
    print("✅ GROQ_API_KEY loaded successfully.")

✅ GROQ_API_KEY loaded successfully.


In [20]:
# Initialize the model
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7)

# Test the connection
response = llm.invoke("Say hello, I’m connected to Groq.")
print("Model response:", response.content)

Model response: Hello, I see you're connected to Groq, a company that provides AI acceleration solutions. How's your experience with Groq so far? Are you working on any exciting projects or exploring the potential of their technology?


If you see a text response above, your connection to Groq works correctly.
We can now move on to understanding **model parameters**, such as `temperature` and `model size`,
to control how the model behaves.


## 2. Temperature and Model Size

Now that our connection works, let’s explore two key model parameters:
- **Model size:** defines the capacity and speed of the model.
  Larger models handle more context but are slower and more expensive.
- **Temperature:** controls randomness and creativity in the generation.
  Lower values → consistent outputs, higher values → more varied answers.

Let’s see how these parameters affect the output.


In [26]:
# We'll use the same model: Llama 3.1 8B Instant
prompt = "Give me a very short feature that I can use in my trading in one sentence"

# Compare two temperature settings
for temp in [0.2, 0.8]:
    llm = ChatGroq(model="llama-3.1-8b-instant", temperature=temp)
    print(f"\n--- Temperature = {temp} ---")
    print(llm.invoke(prompt).content)


--- Temperature = 0.2 ---
The "Golden Cross" feature, which occurs when a short-term moving average (e.g. 50-day) crosses above a long-term moving average (e.g. 200-day), can be a bullish signal indicating a potential long-term trend reversal.

--- Temperature = 0.8 ---
The "Donchian Channel" feature, which involves setting buy or sell signals when the price crosses above or below its highest high or lowest low over a specified period of time (typically 20 days), can be a useful technical indicator for identifying potential trading opportunities.


In [27]:
# We'll use the same model: Llama 3.1 8B Instant
prompt = "Give me a very short feature that I can use in my trading in one sentence"

# Compare two temperature settings
for temp in [0.2, 0.8]:
    llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=temp)
    print(f"\n--- Temperature = {temp} ---")
    print(llm.invoke(prompt).content)


--- Temperature = 0.2 ---
Using a "stop-loss" feature can help limit your potential losses by automatically selling a security when it falls to a certain price, thereby protecting your investment.

--- Temperature = 0.8 ---
Using a "stop-loss" feature, which automatically sells a stock when it falls to a certain price, can help limit potential losses and protect your investments.


As you can see, a higher temperature generates more diverse or unexpected answers,
while a lower temperature gives more deterministic and stable results.

In trading contexts, we often prefer **stability and reproducibility**,
so you’ll see moderate values (around 0.4–0.6) used throughout this course.


## 3. Building a Structured Prompt

So far, we’ve used free text prompts.
Now, we’ll see how **structured prompts** can help the model stay consistent.

LangChain uses two message types:
- **System** → defines the role (context, behavior)
- **User** → gives the actual task to perform



In [28]:
# 3.1. Free text prompt (unstructured)

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7)
response = llm.invoke("Create a simple configuration for a trading bot.")
print(response.content)


**Trading Bot Configuration**

Below is a simple configuration for a trading bot in Python. This configuration includes basic settings such as API keys, trading pairs, and risk management parameters.

### Configuration File (`config.py`)
```python
import os

class Config:
    # API Keys
    BINANCE_API_KEY = os.environ.get('BINANCE_API_KEY')
    BINANCE_API_SECRET = os.environ.get('BINANCE_API_SECRET')

    # Trading Pairs
    PAIRS = [
        ('BTCUSDT', 0.01),  # Bitcoin / USDT with a minimum trade size of 0.01
        ('ETHUSDT', 0.001),  # Ethereum / USDT with a minimum trade size of 0.001
        ('LTCUSDT', 0.001)  # Litecoin / USDT with a minimum trade size of 0.001
    ]

    # Risk Management
    MAX_LEVERAGE = 2
    STOP_LOSS_PCT = 0.05
    TAKE_PROFIT_PCT = 0.05

    # Logging
    LOG_LEVEL = 'INFO'

    # Other Settings
    EXCHANGE = 'BINANCE'
    INTERVAL = '1m'  # 1 minute interval
    START_TIME = '08:00'  # Start trading at 8:00 AM UTC
    END_TIME = '17:00'  # End tr

In [29]:
# 3.2. Structured prompt (system + user)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a quant developer. Always answer in valid YAML."),
    ("user", "Create a configuration for a trading bot with two parameters: window_size=20 and feature='momentum'.")
])

formatted = prompt.format_messages()
response = llm.invoke(formatted)
print(response.content)


```yml
trading_bot:
  parameters:
    window_size: 20
    feature: momentum
```


In [30]:
# 3.3. Structured prompt with JSON output

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a quant developer. Answer strictly in JSON format."),
    ("user", "Give me 3 trading features with a short description each.")
])

formatted = prompt.format_messages()
response = llm.invoke(formatted)
print(response.content)


```json
[
  {
    "name": "Mean Reversion Strategy",
    "description": "A trading strategy that involves buying an asset when its price falls below its historical mean and selling when it rises above, based on the assumption that the asset's price will revert to its mean over time."
  },
  {
    "name": "Trend Following Strategy",
    "description": "A trading strategy that involves identifying and following the direction of a trend in an asset's price, with the goal of profiting from the continuation of the trend."
  },
  {
    "name": "Momentum Trading Strategy",
    "description": "A trading strategy that involves buying an asset when its price is rising rapidly and selling when it is falling rapidly, based on the assumption that the asset's momentum will continue to drive its price higher or lower."
  }
]
```


A good prompt defines:
1. The **role** (system)
2. The **task** (user)
3. The **expected output format**

This combination makes your results predictable, testable, and ready to integrate in automated pipelines.

## 5. Chaining Logic with RunnableSequence

An agent is often made of **several logical steps**.

Instead of writing a long, monolithic process,
we can use LangChain’s `RunnableSequence` to **chain functions together**.

Each step:
1. Receives an input
2. Transforms it
3. Passes it to the next step

Let’s see a simple example.


In [ ]:
# ---------------------------------------------------------------
# Step functions
# ---------------------------------------------------------------
def clean_text(txt: str) -> str:
    """Remove extra spaces and lowercase the text."""
    return txt.strip().lower()

def add_prefix(txt: str) -> str:
    """Add a prefix before the text."""
    return f"Processed: {txt}"

In [34]:
# ---------------------------------------------------------------
# Build the RunnableSequence
# ---------------------------------------------------------------
pipeline = RunnableSequence(
    first=RunnableLambda(clean_text),
    middle=[],
    last=RunnableLambda(add_prefix)
)

In [35]:
# ---------------------------------------------------------------
# Run the sequence
# ---------------------------------------------------------------
output = pipeline.invoke("   Hello Quantreo!   ")
print(output)

Processed: hello quantreo!
